In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [4]:
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 36649892.41it/s]


Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1050002.98it/s]


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 9782453.54it/s]


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 1047595.75it/s]


Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [5]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [11]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 세번째층
        # ImgIn shape=(?, 7, 7, 64)
        #    Conv      ->(?, 7, 7, 128)
        #    Pool      ->(?, 3, 3, 128)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(3 * 3 * 128, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        # out = nn.LogSoftmax(self.fc(out), dim=1) # nn.NLLLoss() 와 매칭
        out = self.fc(out)
        return out

In [13]:
# CNN 모델 정의
model = CNN().to(device) # .to('cuda') == .cuda() / .to('cpu') == .cpu()

In [14]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
# criterion = torch.nn.NLLLoss().to(device)    # nn.LogSoftmax() 와 매칭

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


In [16]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X) # forward 실행
        cost = criterion(hypothesis, Y) # CrossEntropyLoss
        cost.backward() # gradient 계산
        optimizer.step() # weight 업데이트

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.204364821
[Epoch:    2] cost = 0.0482416786
[Epoch:    3] cost = 0.0348232836
[Epoch:    4] cost = 0.0263678413
[Epoch:    5] cost = 0.0211546924
[Epoch:    6] cost = 0.0174575374
[Epoch:    7] cost = 0.0146635436
[Epoch:    8] cost = 0.0111658396
[Epoch:    9] cost = 0.0110918293
[Epoch:   10] cost = 0.00856338535
[Epoch:   11] cost = 0.00855270959
[Epoch:   12] cost = 0.00623815693
[Epoch:   13] cost = 0.00692172442
[Epoch:   14] cost = 0.00637494912
[Epoch:   15] cost = 0.00505425408


In [17]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9884999990463257


# 전체 코드

In [ ]:
import torch # PyTorch 라이브러리를 임포트합니다.
import torchvision.datasets as dsets # torchvision에서 제공하는 데이터셋을 사용하기 위해 임포트합니다.
import torchvision.transforms as transforms # 이미지 변환 도구를 사용하기 위해 임포트합니다.
import torch.nn.init # 신경망 가중치를 초기화하기 위한 함수를 임포트합니다.

device = 'cuda' if torch.cuda.is_available() else 'cpu' # GPU가 사용 가능한 경우 'cuda'를, 그렇지 않은 경우 'cpu'를 사용하도록 설정합니다.

torch.manual_seed(777) # 재현 가능한 결과를 얻기 위해 랜덤 시드를 고정합니다.

if device == 'cuda':
    torch.cuda.manual_seed_all(777) # GPU를 사용하는 경우에도 동일한 랜덤 시드를 사용하도록 설정합니다.

learning_rate = 0.001 # 학습률을 설정합니다.
training_epochs = 15 # 전체 데이터셋에 대한 학습 횟수를 설정합니다.
batch_size = 100 # 미니배치의 크기를 설정합니다.

# MNIST 데이터셋을 다운로드하고 불러옵니다.
mnist_train = dsets.MNIST(root='MNIST_data/', # 데이터셋이 저장될 위치를 지정합니다.
                          train=True, # 학습 데이터셋을 불러옵니다.
                          transform=transforms.ToTensor(), # 이미지를 PyTorch 텐서로 변환합니다.
                          download=True) # 지정된 위치에 데이터셋이 없는 경우 인터넷에서 다운로드합니다.

mnist_test = dsets.MNIST(root='MNIST_data/', # 데이터셋이 저장될 위치를 지정합니다.
                         train=False, # 테스트 데이터셋을 불러옵니다.
                         transform=transforms.ToTensor(), # 이미지를 PyTorch 텐서로 변환합니다.
                         download=True) # 지정된 위치에 데이터셋이 없는 경우 인터넷에서 다운로드합니다.

# 데이터 로더를 생성합니다. 이는 학습 도중에 미니배치를 자동으로 생성해주는 역할을 합니다.
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

# CNN 모델을 정의합니다.
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫 번째 층: 입력 이미지의 채널 수는 1, 출력 채널 수는 32, 커널 크기는 3, 스트라이드는 1, 패딩은 1입니다.
        # ReLU 활성화 함수를 적용한 후, 커널 크기 2의 최대 풀링을 적용합니다.
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두 번째 층: 입력 채널 수는 32, 출력 채널 수는 64, 커널 크기는 3, 스트라이드는 1, 패딩은 1입니다.
        # ReLU 활성화 함수를 적용한 후, 커널 크기 2의 최대 풀링을 적용합니다.
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 세 번째 층: 입력 채널 수는 64, 출력 채널 수는 128, 커널 크기는 3, 스트라이드는 1, 패딩은 1입니다.
        # ReLU 활성화 함수를 적용한 후, 커널 크기 2의 최대 풀링을 적용합니다.
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층: 3*3*128 크기의 입력을 받아 10개의 클래스에 대한 확률을 출력합니다.
        self.fc = torch.nn.Linear(3 * 3 * 128, 10, bias=True)

        # 전결합층의 가중치를 Xavier 초기화를 사용하여 초기화합니다.
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x) # 첫 번째 층의 연산을 수행합니다.
        out = self.layer2(out) # 두 번째 층의 연산을 수행합니다.
        out = self.layer3(out) # 세 번째 층의 연산을 수행합니다.
        out = out.view(out.size(0), -1)   # 전결합층에 넣기 위해 텐서를 펼칩니다.
        out = self.fc(out) # 전결합층의 연산을 수행합니다.
        return out

# CNN 모델을 생성하고 지정한 장치에 할당합니다.
model = CNN().to(device)

# 비용 함수와 최적화 도구를 설정합니다.
criterion = torch.nn.CrossEntropyLoss().to(device) # 비용 함수로 크로스 엔트로피를 사용합니다.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) # 최적화 도구로 Adam을 사용합니다.

total_batch = len(data_loader) # 한 에포크에 필요한 미니배치의 수를 계산합니다.
print('총 배치의 수 : {}'.format(total_batch))

# 모델을 학습합니다.
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니배치 단위로 데이터를 가져옵니다.
        X = X.to(device) # 입력 이미지를 지정한 장치에 할당합니다.
        Y = Y.to(device) # 레이블을 지정한 장치에 할당합니다.

        optimizer.zero_grad() # 기울기를 0으로 초기화합니다.
        hypothesis = model(X) # 모델에 입력을 전달하여 예측값을 계산합니다.
        cost = criterion(hypothesis, Y) # 예측값과 레이블을 비용 함수에 전달하여 비용을 계산합니다.
        cost.backward() # 비용 함수에 대한 기울기를 계산합니다.
        optimizer.step() # 계산한 기울기를 사용하여 모델의 파라미터를 업데이트합니다.

        avg_cost += cost / total_batch # 평균 비용을 계산합니다.

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

# 모델을 평가합니다.
with torch.no_grad(): # 기울기 계산을 비활성화합니다.
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device) # 테스트 이미지를 지정한 장치에 할당합니다.
    Y_test = mnist_test.test_labels.to(device) # 테스트 레이블을 지정한 장치에 할당합니다.

    prediction = model(X_test) # 모델에 테스트 이미지를 전달하여 예측값을 계산합니다.
    correct_prediction = torch.argmax(prediction, 1) == Y_test # 예측값과 레이블을 비교하여 정확도를 계산합니다.
    accuracy = correct_prediction.float().mean() # 정확도를 계산합니다.
    print('Accuracy:', accuracy.item()) # 정확도를 출력합니다.